In [1]:
import numpy as np
from PIL import Image

In [6]:
a, b, step = -2, 2, 0.001 # start, stop, and step values for the 'window'.
N = np.arange(a, b+step, step)
# N_2 = 

def cartesian_product(x, y):
    return np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])

c_0 = cartesian_product(N, N)

print(N)
print(c_0)

# convert the 2D array into an array of complex numbers. 
c_0_i = 1j*c_0[...,1] # build the complex components
c_0_i += c_0[...,0] # build the real components

print(c_0_i)

[-2.    -1.999 -1.998 ...  1.998  1.999  2.   ]
[[-2.    -2.   ]
 [-1.999 -2.   ]
 [-1.998 -2.   ]
 ...
 [ 1.998  2.   ]
 [ 1.999  2.   ]
 [ 2.     2.   ]]
[-2.   -2.j -1.999-2.j -1.998-2.j ...  1.998+2.j  1.999+2.j  2.   +2.j]


In [12]:
# z_(n+1) = z_(n)^2 + c 
# c = some complex number.
# z_0 = 0
# iteration applied repeatedly. 

# try it the basic "dumb" way first!
z = 1j * np.zeros(len(c_0)) # z_0 = 0

# z = 1j * np.random.random(len(c_0)) + np.random.random(len(c_0)) # z_0 = random

for n in range(80):
    z = z**2 + c_0_i
    
print(z[abs(z)<1])
bounded = z[abs(z)<1]
print(len(c_0_i), len(bounded))
print(len(c_0_i)**(1/2))
print(max(abs(z)))

# other ideas: Use quaternions in the formula, visualize in 4D as a video, or reduce to 3 values and visualize in 3D!

/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in square
  if sys.path[0] == '':
/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in square
  if sys.path[0] == '':
/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in less
  


[ 0.04071637-0.10246809j  0.11357783-0.5375051j  -0.18924786-0.75795907j
 ... -0.18619359+0.79047766j  0.00531418-0.39179495j
  0.02198448-0.01684595j]


/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in less
  from ipykernel import kernelapp as app


16008001 833552
4001.0
nan


In [6]:
data = np.zeros( (1024,1024,3), dtype=np.uint8 )

data[512,512] = [254,0,0]       # Makes the middle pixel red
data[512,513] = [0,0,255]       # Makes the next pixel blue

img = Image.fromarray( data )       # Create a PIL image
img.show()    


In [13]:
import matplotlib as plot

# map absolute values to hues.
def color_map(c):
    # IDEA: Use black for values below some cutoff, and hues for values above that. 
    h = np.log(abs(c)) % (360) / (360) # calculate hue.
    hsv = np.column_stack((h, np.ones(len(h)), np.zeros(len(h)) + 0.5))
    rgb = plot.colors.hsv_to_rgb(hsv)
    rgb_int = (rgb * 256).astype(np.uint8) 
    return rgb_int

z_rgb = color_map(z)
print(z_rgb)
# rearrange the 1D array of hue values to 2D.
s = int(len(z_rgb)**(1/2))
img = Image.fromarray(z_rgb.reshape(s,s,3))
img.show()

/home/brian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in remainder
  


[[0 0 0]
 [0 0 0]
 [0 0 0]
 ...
 [0 0 0]
 [0 0 0]
 [0 0 0]]
